In [1]:
import trimesh as tm
import numpy as np
import pyvista as pv
import os
import topogenesis as tg
import pickle as pk

In [5]:
context_path = os.path.relpath('../../data/movedcontext.obj')
context_mesh = tm.load(context_path)

env_lat_path = os.path.relpath('../../data/macrovoxels.csv')
envelope_lattice = tg.lattice_from_csv(env_lat_path)

In [6]:
# create a sphere 
sphere = tm.creation.icosphere(subdivisions= 3, radius= 400.0)
spheremove = sphere.apply_translation([60, -60, 0]) # skydome is moved to correct position
# extract vertices from sphere
sphere_vertices = spheremove.vertices

# keep only positive vertices
skydome = sphere_vertices[sphere_vertices[:,2] > 0.0]

# convert to array
skydome = np.array(skydome)

# shift skydome down to ground height --> shift down with minimum height of skydome points +10 and height offset of lattice and environment
skydome[:,2] += -min(skydome[:,2]-10) + min(envelope_lattice.centroids[:,2])


In [7]:
a = skydome
b = np.array((30,-30,0))
vec = b - a
c = tm.creation.icosphere(subdivisions= 1, radius= 400.0)
vec.shape, skydome.shape, sphere_vertices.shape, c.vertices.shape

((305, 3), (305, 3), (642, 3), (42, 3))

In [8]:
p = pv.Plotter(notebook=True)

def tri_to_pv(tri_mesh):
    faces = np.pad(tri_mesh.faces, ((0, 0),(1,0)), 'constant', constant_values=3)
    pv_mesh = pv.PolyData(tri_mesh.vertices, faces)
    return pv_mesh

# fast visualization of the lattice
#envelope_lattice.fast_vis(p)

# add the sky vectors
#p.add_points(skydome, color='#0013ff')
p.add_arrows(skydome, vec, mag=0.1, show_scalar_bar=False)


# add context
#p.add_mesh(tri_to_pv(context_mesh), opacity=0.1, color='lightgrey')


# plotting
p.show(use_ipyvtk=True, screenshot='skydome.png')

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

[(1375.1069231023332, 1255.1069231023332, 1477.5763044347307),
 (60.0, -60.0, 162.46938133239746),
 (0.0, 0.0, 1.0)]

In [ ]:
# save skydome to pk
pk.dump(skydome, open("../../data/skydome.pk", "wb"))